In [1]:
import json
from tqdm import tqdm
from openai import OpenAI
import re
import time

In [ ]:
def virtual_patient(msgs):
    api_key = ""
    base_url = ""

    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    chat_completion = client.chat.completions.create(
        messages=msgs,
        model="deepseek-v3",
        temperature=0.1,
        presence_penalty=0,
        frequency_penalty=0,
        top_p=1
    )
    return chat_completion

In [ ]:
def dialogue_monitor(msgs):
    api_key = ""
    base_url = ""

    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    chat_completion = client.chat.completions.create(
        messages=msgs,
        model="deepseek-v3",
        max_tokens=10,
        temperature=0.1,
        presence_penalty=0,
        frequency_penalty=0,
        top_p=1
    )
    return chat_completion

In [ ]:
def doctor_r1(msgs):
    api_key = ""
    base_url = ""

    client = OpenAI(
        api_key=api_key,
        base_url=base_url
    )

    chat_completion = client.chat.completions.create(
        messages=msgs,
        model="deepseek-r1",
        temperature=0.1,
        presence_penalty=0,
        frequency_penalty=0,
        top_p=1
    )
    return chat_completion

In [5]:
def gen_group_chat_prompt(group_chat):
    prompt = ""
    for data in group_chat:
        prompt += f"{data['role']}:{data['content']}\n"
    return prompt

In [6]:
def gen_vp_prompt(vp_info):
    prompt_list = []
    for question in vp_info['question_list']:
        prompt_list.append([vp_info['ehr_prompt'], "\n患者核心诉求：" + json.dumps(question, ensure_ascii=False)])
    return prompt_list

In [7]:
vp_info = json.load(open('../../dataset/vp_info.json', 'r'))
vp_prompt_list = []
for data in vp_info:
    vp_prompt_list.extend(gen_vp_prompt(data))

In [8]:
group_chat = json.load(open('../../dataset/group_chat.json', 'r'))
group_chat_prompt = gen_group_chat_prompt(group_chat[:1000])

In [9]:
def get_recode(role, response, inference_time):
    return {
        "role": role,
        "content": response.choices[0].message.content,
        "completion_tokens": response.usage.completion_tokens,
        "prompt_tokens": response.usage.prompt_tokens,
        "inference_time": inference_time
    }

In [ ]:
# recoder_list = []
recoder_list = json.load(open('../../output/r1_conv.json', 'r'))

for vp_prompt in tqdm(vp_prompt_list[len(recoder_list)]):
    recoder = {
        "vp_prompt": vp_prompt[0] + vp_prompt[1],
        "conversation": []
    }

    dialogue_monitor_msgs = [{
        "role": "system",
        "content": "你是一名对话内容审查员，功能是判断对话是否已经完成，你只需要输出True或False即可"
    },
    {
        "role": "user",
        "content": vp_prompt[1] + "请根据**患者的核心诉求**，判断下面对话中患者是否已经获取到了所有需要的信息，如果是则输出True，否则输出False，不需要输出其他内容。"
    },
    {
        "role": "user",
        "content": "对话内容："
    }]

    virtual_patient_msgs = [{
        "role": "system",
        "content": "现在你是一名乳腺癌术后患者，你的患者信息和核心诉求在上述内容中已被明确定义。目前的场景为你正在群聊当中向医护人员寻求帮助。"
    },
    {
        "role": "user",
        "content": f"乳腺癌术后康复群群聊示例：\n{group_chat_prompt}\n\n" + vp_prompt[0] + vp_prompt[1] + "现在你是一名乳腺癌术后患者，你的患者信息和核心诉求在上述内容中已被明确定义。目前的场景为你正在群聊当中向医护人员寻求帮助。请仿照'乳腺癌术后康复群群聊示例'中患者交流的方式，表达你的诉求。注意：每次只需要说出一个诉求，不一定要按顺序说出诉求，对话过程注意关联上下文和你的个人信息，尽可能能够对我说的话作出一些反馈。在满足核心诉求后可以礼貌结束对话。我将作为医生与你对话，你只需要扮演患者即可。"
    }
    ]

    doctor_msgs = [{
        "role": "system",
        "content": "现在你是一名乳腺癌专科医师，你的患者在手术后联系你获取帮助，该患者的信息如下：" + vp_prompt[0] + "\n请结合患者信息给出尽可能专业有效并且容易理解的反馈。"
    }]

    count = 0

    while 1:
        # generate virtual patient's content
        start_time = time.time()
        vp_response = virtual_patient(virtual_patient_msgs)
        end_time = time.time()
        vp_content = vp_response.choices[0].message.content
        recoder['conversation'].append(get_recode("virtual patient", vp_response, end_time-start_time))
        
        dialogue_monitor_msgs[2]['content'] += f"患者：{vp_content}"
        virtual_patient_msgs.append({
            "role": "assistant",
            "content": vp_content
        })
        doctor_msgs.append({
            "role": "user",
            "content": vp_content
        })

        # generate doctor's content
        start_time = time.time()
        doctor_response = doctor_r1(doctor_msgs)
        end_time = time.time()
        doctor_content = re.sub(r"<think>.*?</think>", "", doctor_response.choices[0].message.content, flags=re.DOTALL)
        recoder['conversation'].append(get_recode("doctor", doctor_response, end_time-start_time))

        dialogue_monitor_msgs[2]['content'] += f"医生：{doctor_content}"
        virtual_patient_msgs.append({
            "role": "user",
            "content": doctor_content
        })
        doctor_msgs.append({
            "role": "assistant",
            "content": doctor_content
        })

        # decide whether stop conversation
        monitor_response = dialogue_monitor(dialogue_monitor_msgs)
        monitor_content = monitor_response.choices[0].message.content
        if 'True' in monitor_content or 'true' in monitor_content:
            recoder_list.append(recoder)
            break
        else:
            count += 1
            if count > 6:
                print("Too many turns. Force to stop!")
                recoder_list.append(recoder)
                break

In [ ]:
error_id = [36, 56, 63, 69, 71, 74, 76, 77, 79, 81, 82, 83, 87, 89, 90, 96]
recoder_list = []

for id in tqdm(error_id):
    vp_prompt = vp_prompt_list[id]

    recoder = {
        "vp_prompt": vp_prompt[0] + vp_prompt[1],
        "conversation": []
    }

    dialogue_monitor_msgs = [{
        "role": "system",
        "content": "你是一名对话内容审查员，功能是判断对话是否已经完成，你只需要输出True或False即可"
    },
    {
        "role": "user",
        "content": vp_prompt[1] + "请根据**患者的核心诉求**，判断下面对话中患者是否已经获取到了所有需要的信息，如果是则输出True，否则输出False，不需要输出其他内容。"
    },
    {
        "role": "user",
        "content": "对话内容："
    }]

    virtual_patient_msgs = [{
        "role": "system",
        "content": "现在你是一名乳腺癌术后患者，你的患者信息和核心诉求在上述内容中已被明确定义。目前的场景为你正在群聊当中向医护人员寻求帮助。"
    },
    {
        "role": "user",
        "content": f"乳腺癌术后康复群群聊示例：\n{group_chat_prompt}\n\n" + vp_prompt[0] + vp_prompt[1] + "现在你是一名乳腺癌术后患者，你的患者信息和核心诉求在上述内容中已被明确定义。目前的场景为你正在群聊当中向医护人员寻求帮助。请仿照'乳腺癌术后康复群群聊示例'中患者交流的方式，表达你的诉求。注意：每次只需要说出一个诉求，不一定要按顺序说出诉求，对话过程注意关联上下文和你的个人信息，尽可能能够对我说的话作出一些反馈。在满足核心诉求后可以礼貌结束对话。我将作为医生与你对话，你只需要扮演患者即可。"
    }
    ]

    doctor_msgs = [{
        "role": "system",
        "content": "现在你是一名乳腺癌专科医师，你的患者在手术后联系你获取帮助，该患者的信息如下：" + vp_prompt[0] + "\n请结合患者信息给出尽可能专业有效并且容易理解的反馈。"
    }]

    count = 0

    while 1:
        # generate virtual patient's content
        start_time = time.time()
        vp_response = virtual_patient(virtual_patient_msgs)
        end_time = time.time()
        vp_content = vp_response.choices[0].message.content
        recoder['conversation'].append(get_recode("virtual patient", vp_response, end_time-start_time))
        
        dialogue_monitor_msgs[2]['content'] += f"患者：{vp_content}"
        virtual_patient_msgs.append({
            "role": "assistant",
            "content": vp_content
        })
        doctor_msgs.append({
            "role": "user",
            "content": vp_content
        })

        # generate doctor's content
        start_time = time.time()
        doctor_response = doctor_r1(doctor_msgs)
        end_time = time.time()
        doctor_content = re.sub(r"<think>.*?</think>", "", doctor_response.choices[0].message.content, flags=re.DOTALL)
        recoder['conversation'].append(get_recode("doctor", doctor_response, end_time-start_time))

        dialogue_monitor_msgs[2]['content'] += f"医生：{doctor_content}"
        virtual_patient_msgs.append({
            "role": "user",
            "content": doctor_content
        })
        doctor_msgs.append({
            "role": "assistant",
            "content": doctor_content
        })

        # decide whether stop conversation
        monitor_response = dialogue_monitor(dialogue_monitor_msgs)
        monitor_content = monitor_response.choices[0].message.content
        if 'True' in monitor_content or 'true' in monitor_content:
            recoder_list.append(recoder)
            print(id)
            print(json.dumps(recoder,ensure_ascii=False))
            break
        else:
            count += 1
            if count > 5:
                print("Too many turns. Force to stop!")
                print(id)
                print(json.dumps(recoder,ensure_ascii=False))
                break

  0%|          | 0/16 [00:00<?, ?it/s]

  6%|▋         | 1/16 [05:08<1:17:05, 308.34s/it]

Too many turns. Force to stop!
36
{"vp_prompt": "患者信息：年龄：44，月经情况：有，确诊手段：组织学病理，肿块位置：左乳，肿块部位：外下，查体长径(cm)：2.0，查体短径(cm)：未知，B超长径(cm)：1.9，B超短径(cm)：1.38，淋巴结肿大：无，淋巴结穿刺：无，穿刺结果：未检，临床分期T：1，临床分期N：0，临床分期M：0，情况备注：MRI未诉肿块大小;ICH：ER<5%，余阴性，手术性质：根治性，手术方式：保乳+腋清，病灶长径(cm)：4.0，病灶短径(cm)：3.8，组织学分类：浸润性导管癌，转移淋巴结个数：0.0，总清扫淋巴结个数：3.0，前哨淋巴结定位：有，脉管内癌栓：无，神经侵犯：无，癌结节形成：无，免疫组化：有，ER：阴性，PR：阴性，Her2分型：HerHer2-Positive-Zero，Ki67：>50%，分子分型：三阴性，@21基因RS分数：未知，病理分期T：2，病理分期N：0，病理分期M：0，病理分期：Stage IIA，病理备注：未知，化疗时机：术后辅助化疗，化疗次数术前：未知，化疗次数术后：8.0，化疗方案术前：未行，化疗方案术后：AC-T，化疗方案备注：未知，新辅助化疗疗效评估：未行，放疗：有，内分泌药物：未知，药物备注：未知，分子靶向药物：无，\n患者核心诉求：[\"疾病的发展趋势如何？\", \"术后如何改善睡眠质量？\", \"术后性生活如何恢复？\"]", "conversation": [{"role": "virtual patient", "content": "护士长，您好，我今年44岁，刚做完乳腺癌保乳手术，现在术后恢复中。最近睡眠质量特别差，经常失眠，想问一下术后如何改善睡眠质量？有没有什么好的建议或者可以帮助睡眠的药物？谢谢！", "completion_tokens": 114, "prompt_tokens": 36172, "inference_time": 9.849940061569214}, {"role": "doctor", "content": "<think>\n好的，我现在需要帮助这位乳腺癌术后患者解决失眠的问题。首先，我需要回顾她的病史和治疗情况，确保我的建议不会与她的治疗冲突。她44岁，接受了保乳手术和腋窝淋巴结清扫，术后病理显示

 12%|█▎        | 2/16 [09:42<1:07:15, 288.27s/it]

Too many turns. Force to stop!
56
{"vp_prompt": "患者信息：年龄：46，月经情况：有，确诊手段：细胞学病理，肿块位置：左乳，肿块部位：外上，查体长径(cm)：5.0，查体短径(cm)：5.0，B超长径(cm)：2.37，B超短径(cm)：1.6，淋巴结肿大：有，淋巴结穿刺：无，穿刺结果：未检，临床分期T：2，临床分期N：1，临床分期M：0，情况备注：未知，手术性质：根治性，手术方式：改良根治，病灶长径(cm)：3.0，病灶短径(cm)：1.8，组织学分类：浸润性导管癌，转移淋巴结个数：2.0，总清扫淋巴结个数：24.0，前哨淋巴结定位：无，脉管内癌栓：无，神经侵犯：无，癌结节形成：无，免疫组化：有，ER：阳性，PR：阳性，Her2分型：HerHer2-Positive-Zero，Ki67：>15%, <=50%，分子分型：Luminal-B HER-Luminal-B HER-2阳性阴性，@21基因RS分数：未知，病理分期T：2，病理分期N：1，病理分期M：0，病理分期：Stage IIB，病理备注：未知，化疗时机：术后辅助化疗，化疗次数术前：0.0，化疗次数术后：6.0，化疗方案术前：未行，化疗方案术后：FEC，化疗方案备注：未知，新辅助化疗疗效评估：未行，放疗：无，内分泌药物：托瑞米芬，药物备注：未知，分子靶向药物：无，\n患者核心诉求：[\"疾病的发展趋势如何？\", \"术后如何改善睡眠质量？\", \"术后性生活如何恢复？\"]", "conversation": [{"role": "virtual patient", "content": "护士长您好，我是46岁，左乳浸润性导管癌术后患者，目前正在服用托瑞米芬进行内分泌治疗。我最近睡眠质量不太好，晚上经常醒来，早上起来感觉特别累。请问这种情况该怎么改善呢？我看到群里有人说褪黑素，我适合吃吗？还是说有其他更好的方法？", "completion_tokens": 145, "prompt_tokens": 36175, "inference_time": 14.290918588638306}, {"role": "doctor", "content": "<think>\n好的，我现在需要处理一位乳腺癌术后患者的咨询。患者46岁，已经接受了左乳改良根治术

...

Too many turns. Force to stop!
79
{"vp_prompt": "患者信息：年龄：58，月经情况：无，确诊手段：组织学病理，肿块位置：右乳，肿块部位：外下，查体长径(cm)：10.0，查体短径(cm)：8.0，B超长径(cm)：未知，B超短径(cm)：未知，淋巴结肿大：有，淋巴结穿刺：无，穿刺结果：未检，临床分期T：4，临床分期N：1，临床分期M：0，情况备注：外院穿刺明确，手术性质：根治性，手术方式：改良根治，病灶长径(cm)：4.0，病灶短径(cm)：3.5，组织学分类：其他，转移淋巴结个数：9.0，总清扫淋巴结个数：34.0，前哨淋巴结定位：无，脉管内癌栓：有，神经侵犯：无，癌结节形成：无，免疫组化：有，ER：阳性，PR：阳性，Her2分型：HerHer2-Positive-Low，Ki67：<=15%，分子分型：Luminal-A，@21基因RS分数：未知，病理分期T：4，病理分期N：2，病理分期M：0，病理分期：Stage IIIB，病理备注：化疗后改变，示侵润癌，化疗时机：新辅助化疗，化疗次数术前：3.0，化疗次数术后：4.0，化疗方案术前：未行，化疗方案术后：未行，化疗方案备注：术前三次VCR，术后4次CMF，新辅助化疗疗效评估：PR，放疗：有，内分泌药物：他莫替芬，药物备注：未知，分子靶向药物：无，\n患者核心诉求：[\"病理报告中的指标如何解读？\", \"术后如何恢复体力？\", \"如何应对术后孤独感？\"]", "conversation": [{"role": "virtual patient", "content": "@护士长 您好,我最近刚做完右乳改良根治手术,病理报告出来了,但对里面的指标不太理解,想请教一下。特别是分子分型Luminal-A是什么意思?ER、PR阳性又代表什么?这些指标对后续治疗有影响吗?谢谢护士长的解答!🌹", "completion_tokens": 129, "prompt_tokens": 36173, "inference_time": 8.466946840286255}, {"role": "doctor", "content": "<think>\n嗯，我现在需要处理这位患者的问题。她刚刚做完右乳改良根治手术，现在对病理报告中的一些指标有疑问，特别是关于分子分型Lum

 62%|██████▎   | 10/16 [46:39<29:41, 296.91s/it]

Too many turns. Force to stop!
81
{"vp_prompt": "患者信息：年龄：56，月经情况：无，确诊手段：组织学病理，肿块位置：左乳，肿块部位：外上，查体长径(cm)：2.5，查体短径(cm)：未知，B超长径(cm)：2.1，B超短径(cm)：1.6，淋巴结肿大：有，淋巴结穿刺：无，穿刺结果：未检，临床分期T：2，临床分期N：2，临床分期M：0，情况备注：双乳癌，2006.2.2发现右乳癌伴锁骨上淋巴结转移，手术性质：根治性，手术方式：改良根治，病灶长径(cm)：1.5，病灶短径(cm)：1.5，组织学分类：浸润性导管癌，转移淋巴结个数：未知，总清扫淋巴结个数：未知，前哨淋巴结定位：无，脉管内癌栓：无，神经侵犯：无，癌结节形成：有，免疫组化：有，ER：阴性，PR：阴性，Her2分型：HerHer2-Positive-Low，Ki67：>15%, <=50%，分子分型：三阴性，@21基因RS分数：未知，病理分期T：1，病理分期N：3，病理分期M：0，病理分期：Stage IIIC，病理备注：左腋下4个癌结节，化疗时机：新辅助化疗，化疗次数术前：4.0，化疗次数术后：4.0，化疗方案术前：未行，化疗方案术后：FEC，化疗方案备注：后两次＋艾素，单独艾素，顺铂，吉西他滨等，新辅助化疗疗效评估：未行，放疗：有，内分泌药物：未知，药物备注：未知，分子靶向药物：无，\n患者核心诉求：[\"病理报告中的指标代表什么？\", \"如何有效管理术后疼痛？\", \"如何应对术后抑郁情绪？\"]", "conversation": [{"role": "virtual patient", "content": "护士长您好，我是上周刚做完左乳改良根治手术的患者，现在术后感觉伤口处有些疼痛，尤其是晚上更明显，想请教一下如何有效管理术后疼痛？我现在每天按时吃医生开的止痛药，但效果不是很理想。", "completion_tokens": 121, "prompt_tokens": 36222, "inference_time": 13.87610149383545}, {"role": "doctor", "content": "<think>\n好的，我现在需要处理一个乳腺癌术后患者关于疼痛管理的问题。首先，患者的基本情况我得先理清楚。她56岁，已经

 69%|██████▉   | 11/16 [50:33<23:07, 277.57s/it]

82
{"vp_prompt": "患者信息：年龄：56，月经情况：无，确诊手段：组织学病理，肿块位置：左乳，肿块部位：外上，查体长径(cm)：2.5，查体短径(cm)：未知，B超长径(cm)：2.1，B超短径(cm)：1.6，淋巴结肿大：有，淋巴结穿刺：无，穿刺结果：未检，临床分期T：2，临床分期N：2，临床分期M：0，情况备注：双乳癌，2006.2.2发现右乳癌伴锁骨上淋巴结转移，手术性质：根治性，手术方式：改良根治，病灶长径(cm)：1.5，病灶短径(cm)：1.5，组织学分类：浸润性导管癌，转移淋巴结个数：未知，总清扫淋巴结个数：未知，前哨淋巴结定位：无，脉管内癌栓：无，神经侵犯：无，癌结节形成：有，免疫组化：有，ER：阴性，PR：阴性，Her2分型：HerHer2-Positive-Low，Ki67：>15%, <=50%，分子分型：三阴性，@21基因RS分数：未知，病理分期T：1，病理分期N：3，病理分期M：0，病理分期：Stage IIIC，病理备注：左腋下4个癌结节，化疗时机：新辅助化疗，化疗次数术前：4.0，化疗次数术后：4.0，化疗方案术前：未行，化疗方案术后：FEC，化疗方案备注：后两次＋艾素，单独艾素，顺铂，吉西他滨等，新辅助化疗疗效评估：未行，放疗：有，内分泌药物：未知，药物备注：未知，分子靶向药物：无，\n患者核心诉求：[\"复发风险有多高？\", \"术后饮食有什么注意事项？\", \"如何克服对复发的恐惧？\"]", "conversation": [{"role": "virtual patient", "content": "护士长，我是左乳癌术后患者，想请问一下我这种情况的复发风险有多高？我是三阴性乳腺癌，病理分期是 Stage IIIC，化疗和放疗都做了，但还是有点担心。", "completion_tokens": 92, "prompt_tokens": 36217, "inference_time": 9.51104187965393}, {"role": "doctor", "content": "<think>\n好的，我需要仔细分析这位患者的情况，然后回答她的复发风险问题。首先，她提到是左乳癌术后，三阴性乳腺癌，病理分期IIIC，已经完成了化疗和放疗。但她仍然担心复发。\n\n首先，我要回忆一下三阴性乳腺癌的特点。

In [ ]:
print(json.dumps(recoder,ensure_ascii=False))

{"vp_prompt": "患者信息：年龄：44，月经情况：有，确诊手段：组织学病理，肿块位置：左乳，肿块部位：外下，查体长径(cm)：2.0，查体短径(cm)：未知，B超长径(cm)：1.9，B超短径(cm)：1.38，淋巴结肿大：无，淋巴结穿刺：无，穿刺结果：未检，临床分期T：1，临床分期N：0，临床分期M：0，情况备注：MRI未诉肿块大小;ICH：ER<5%，余阴性，手术性质：根治性，手术方式：保乳+腋清，病灶长径(cm)：4.0，病灶短径(cm)：3.8，组织学分类：浸润性导管癌，转移淋巴结个数：0.0，总清扫淋巴结个数：3.0，前哨淋巴结定位：有，脉管内癌栓：无，神经侵犯：无，癌结节形成：无，免疫组化：有，ER：阴性，PR：阴性，Her2分型：HerHer2-Positive-Zero，Ki67：>50%，分子分型：三阴性，@21基因RS分数：未知，病理分期T：2，病理分期N：0，病理分期M：0，病理分期：Stage IIA，病理备注：未知，化疗时机：术后辅助化疗，化疗次数术前：未知，化疗次数术后：8.0，化疗方案术前：未行，化疗方案术后：AC-T，化疗方案备注：未知，新辅助化疗疗效评估：未行，放疗：有，内分泌药物：未知，药物备注：未知，分子靶向药物：无，\n患者核心诉求：[\"疾病的发展趋势如何？\", \"术后如何改善睡眠质量？\", \"术后性生活如何恢复？\"]", "conversation": [{"role": "virtual patient", "content": "护士长您好，我现在做完乳腺癌保乳手术已经一个多月了，但晚上总是睡不好，翻来覆去到凌晨才能睡着，第二天又特别困。请问有什么办法可以改善睡眠质量吗？我听说褪黑素有用，能用这个吗？", "completion_tokens": 116, "prompt_tokens": 36172, "inference_time": 13.949546575546265}, {"role": "doctor", "content": "<think>\n好的，我需要处理这位患者关于睡眠问题的咨询。首先，用户是44岁的女性，刚做完乳腺癌保乳手术加腋窝淋巴结清扫，术后一个多月，现在主诉夜间失眠，入睡困难到凌晨，白天疲倦。她询问改善睡眠的方法，并考虑使用褪黑素。\n\n首先，我要回顾她的病史。她的

In [ ]:
# json.dump(recoder_list, open('../../output/r1_conv_raw.json', 'w'), ensure_ascii=False)